In [ ]:
import sys
import os
# to import quask, move from docs/source/notebooks to src
sys.path.append('../../../src')

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [16]:
import quask
from quask.core import Ansatz, Kernel, KernelFactory, KernelType
from quask.core_implementation import QiskitKernel

In [17]:
qX1 = np.load('Anomaly_Detection_Protons/background_subsampled.npy')
qX2 = np.load('Anomaly_Detection_Protons/signal_AtoH_to_ZZZ_subsampled.npy')

In [18]:
qX1 = qX1[:10,:]
qX2 = qX2[:10,:]

In [19]:
qX = np.row_stack([qX1, qX2])
qy = np.array([-1] * len(qX1) + [1] * len(qX2))

print(f"{qX.shape=}")
print(f"{qy.shape=}")

qX.shape=(20, 8)
qy.shape=(20,)


In [50]:
qX_train, qX_test, qy_train, qy_test = train_test_split(qX, qy, test_size=0.2, random_state=42)

print("Shape training set:", qX_train.shape, qy_train.shape)
print("Shape testing set:", qX_test.shape, qy_test.shape)
print("Example of feature:", qX_train[0])
print("Example of label:", qy_train[0])

Shape training set: (16, 8) (16,)
Shape testing set: (4, 8) (4,)
Example of feature: [-0.23470192 -0.49848634 -0.13048592  0.6318868  -0.5250736  -0.6175051
  0.08415551  0.43143862]
Example of label: -1


In [52]:
def create_qiskit_noiseless(ansatz: Ansatz, measurement: str, type: KernelType):
    return QiskitKernel(ansatz, measurement, type, platform="infty_shots")

# KernelFactory.add_implementation('qiskit_noiseless', create_qiskit_noiseless)
KernelFactory.set_current_implementation('qiskit_noiseless')

In [53]:
# Create a quantum kernel
ansatz = Ansatz(n_features=8, n_qubits=2, n_operations=1)
ansatz.initialize_to_identity()
ansatz.change_operation(0, new_feature=0, new_wires=[0, 1], new_generator="XX", new_bandwidth=0.1)

# ansatz.change_operation(1, new_feature=1, new_wires=[1, 2], new_generator="XY", new_bandwidth=0.1)
# ansatz.change_operation(2, new_feature=2, new_wires=[2, 3], new_generator="XZ", new_bandwidth=0.1)
# ansatz.change_operation(3, new_feature=3, new_wires=[3, 0], new_generator="YX", new_bandwidth=0.1)
# ansatz.change_operation(4, new_feature=4, new_wires=[0, 1], new_generator="YY", new_bandwidth=0.1)
# ansatz.change_operation(5, new_feature=5, new_wires=[1, 2], new_generator="YZ", new_bandwidth=0.1)
# ansatz.change_operation(6, new_feature=6, new_wires=[2, 3], new_generator="ZX", new_bandwidth=0.1)
# ansatz.change_operation(7, new_feature=7, new_wires=[3, 0], new_generator="ZY", new_bandwidth=0.1)
# kernel = KernelFactory.create_kernel(ansatz, "ZZZZ", KernelType.FIDELITY)

In [54]:
kernel = KernelFactory.create_kernel(ansatz, "ZZ", KernelType.FIDELITY)

In [55]:
from quask.core_implementation import QiskitKernel
kernel = QiskitKernel(ansatz, "Z" * 2, KernelType.FIDELITY, platform="infty_shots")

In [56]:
print(kernel.ansatz.n_features)

8


In [24]:
# x1 = np.array([0.001])
# x2 = np.array([0.999])
# similarity = kernel.kappa(x1, x2)
# print("The kernel value between x1 and x2 is", similarity)

In [57]:
# Create the kernel Gram matrices
K_train = kernel.build_kernel(qX_train, qX_train)
K_test = kernel.build_kernel(qX_test, qX_train)

In [ ]:
print(K_train)
result = kernel.kappa(X_train[0], X_train[0])
print("The kernel value between x1 and x2 is", similarity)

None


ValueError: Mismatching number of values and parameters. For partial binding please pass a mapping of {parameter: value} pairs.

In [ ]:
# Fit the model to the training data
model = SVC(kernel='precomputed')
model.fit(K_train, qy_train)

ValueError: Expected 2D array, got scalar array instead:
array=nan.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# Test the model and calculate the score
y_pred = model.predict(K_test)
accuracy = np.sum(qy_test == y_pred) / len(qy_test)
print("Accuracy:", accuracy)